In [3]:
from aidream_data.core import AidreamDatabase
from aidream_data import constants

from pathlib import Path
from tqdm import tqdm
import pandas as pd
import numpy as np
import shutil
import os


# 0. Load Aidream_database, list of perfusion patients : 

In [4]:
# Load the AIDREAM Database : 
aidream_db = AidreamDatabase()
aidream_db.connect()


In [5]:
# Load the perfusion patients list : 
with open("list_patients.txt", 'r') as f:
    list_patients = f.readlines()

list_patients = [p.strip('\n') for p in list_patients]
    

In [6]:
def sort_name_list(list_patients):

    sorted_list = [list_patients[i] for i in np.argsort([int(p.strip("AIDREAM_")) for p in list_patients])]
    return sorted_list
    

In [7]:
dir_aidream_data = Path("/media/maichi/T9/AIDREAM_DATA")


# 1. Load MRI DICOM data, for the native ref : 

In [6]:
for patient in tqdm(list_patients):
    for stage in ['pre_RT', 'Rechute']:

        dir_labels = dir_aidream_data / patient / stage / 'Labels'

        if dir_labels.exists():
            shutil.rmtree(dir_labels)
            

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 234/234 [00:00<00:00, 3062.48it/s]


In [7]:
list_bad_patients = {"AIDREAM_32", "AIDREAM_61"}


In [8]:
# export MRI images in native ref for both pre_RT and Rechute  : 
aidream_dataset = aidream_db.create_dataset(list_aidream_ids=list(set(list_patients) - list_bad_patients),
                                            list_pre_rt=['T1', 'T1CE', 'FLAIR', 'L1', 'L3', 'CTH', 'rCBV'],
                                            list_rechute=['T1', 'T1CE', 'FLAIR', 'L3'],
                                            load_anyway=True,
                                           )


In [9]:
aidream_dataset.export(dir_dataset="/media/maichi/T9/AIDREAM_DATA",
                       overwrite=False)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 232/232 [00:32<00:00,  7.06it/s]


# 2. Load PERFUSION data : 

In [9]:
dir_src_perfusion = Path("/media/maichi/SSD-IGR/Perfusion data")


In [10]:
remaining_patients = set(list_patients) - set(os.listdir(dir_src_perfusion))

In [13]:
sort_name_list(list(remaining_patients))

['AIDREAM_5',
 'AIDREAM_25',
 'AIDREAM_30',
 'AIDREAM_40',
 'AIDREAM_42',
 'AIDREAM_43',
 'AIDREAM_53',
 'AIDREAM_57',
 'AIDREAM_63',
 'AIDREAM_70',
 'AIDREAM_74',
 'AIDREAM_78',
 'AIDREAM_121',
 'AIDREAM_125',
 'AIDREAM_131',
 'AIDREAM_144',
 'AIDREAM_146',
 'AIDREAM_226',
 'AIDREAM_231',
 'AIDREAM_235',
 'AIDREAM_239',
 'AIDREAM_248',
 'AIDREAM_303',
 'AIDREAM_324',
 'AIDREAM_330',
 'AIDREAM_334',
 'AIDREAM_335',
 'AIDREAM_339',
 'AIDREAM_351',
 'AIDREAM_354',
 'AIDREAM_362',
 'AIDREAM_363',
 'AIDREAM_366',
 'AIDREAM_375',
 'AIDREAM_389',
 'AIDREAM_397']

In [11]:
for patient in tqdm(list_patients):

    try:
        dir_src = dir_src_perfusion / patient 
        
        dir_dst = dir_aidream_data / patient / "pre_RT" / "PERFUSION" / "DICOM"
        dir_dst.parent.mkdir(parents=True, exist_ok=True)

        if not dir_dst.exists():
            shutil.copytree(src=dir_src, dst=dir_dst)

    except Exception as e:
        print(fr"{patient} has no PERFUSION data !")
        

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 234/234 [00:00<00:00, 84044.11it/s]

AIDREAM_5 has no PERFUSION data !
AIDREAM_25 has no PERFUSION data !
AIDREAM_30 has no PERFUSION data !
AIDREAM_40 has no PERFUSION data !
AIDREAM_42 has no PERFUSION data !
AIDREAM_43 has no PERFUSION data !
AIDREAM_53 has no PERFUSION data !
AIDREAM_57 has no PERFUSION data !
AIDREAM_63 has no PERFUSION data !
AIDREAM_70 has no PERFUSION data !
AIDREAM_74 has no PERFUSION data !
AIDREAM_78 has no PERFUSION data !
AIDREAM_121 has no PERFUSION data !
AIDREAM_125 has no PERFUSION data !
AIDREAM_131 has no PERFUSION data !
AIDREAM_144 has no PERFUSION data !
AIDREAM_146 has no PERFUSION data !
AIDREAM_226 has no PERFUSION data !
AIDREAM_231 has no PERFUSION data !
AIDREAM_235 has no PERFUSION data !
AIDREAM_239 has no PERFUSION data !
AIDREAM_248 has no PERFUSION data !
AIDREAM_303 has no PERFUSION data !
AIDREAM_324 has no PERFUSION data !
AIDREAM_330 has no PERFUSION data !
AIDREAM_334 has no PERFUSION data !
AIDREAM_335 has no PERFUSION data !
AIDREAM_339 has no PERFUSION data !
AIDRE

In [12]:
list_patients_interim = os.listdir(dir_src_perfusion)
print(fr"Number of patients with available PERFUSION data (for now) : {len(list_patients_interim)}")


Number of patients with available PERFUSION data (for now) : 198


In [13]:
with open("list_intermediary_patients.txt", "w") as f:
    f.writelines([f"{p}\n" for p in list_patients])
    

# 3. Load the CERCARE Data : 

In [14]:
dir_src_cercare = Path("/media/maichi/T7/AIDREAM_DATA/raw")


In [15]:
list_patients_interim = set(list_patients)


In [16]:
for patient in tqdm(list_patients):
    for imaging in ['CTH', 'OEF', 'rCBV', 'Delay', 'COV', 'rCMRO2', 'rLeakage']:
        
        try:
            dir_src = dir_src_cercare / patient / "pre_RT" / imaging
            
            dir_dst = dir_aidream_data / patient / "pre_RT" / imaging / "DICOM"
            dir_dst.parent.mkdir(parents=True, exist_ok=True)

            if not dir_dst.exists():
                shutil.copytree(src=dir_src, dst=dir_dst)            
    
        except Exception as e:

            list_patients_interim = list_patients_interim - {patient}
            print(fr"{patient} has no {imaging} data !")   
    

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 234/234 [00:00<00:00, 11040.13it/s]

AIDREAM_5 has no CTH data !
AIDREAM_5 has no OEF data !
AIDREAM_5 has no rCBV data !
AIDREAM_5 has no Delay data !
AIDREAM_5 has no COV data !
AIDREAM_5 has no rCMRO2 data !
AIDREAM_5 has no rLeakage data !
AIDREAM_10 has no OEF data !
AIDREAM_10 has no Delay data !
AIDREAM_10 has no COV data !
AIDREAM_10 has no rCMRO2 data !
AIDREAM_10 has no rLeakage data !
AIDREAM_15 has no OEF data !
AIDREAM_15 has no Delay data !
AIDREAM_15 has no COV data !
AIDREAM_15 has no rCMRO2 data !
AIDREAM_15 has no rLeakage data !
AIDREAM_16 has no OEF data !
AIDREAM_16 has no Delay data !
AIDREAM_16 has no COV data !
AIDREAM_16 has no rCMRO2 data !
AIDREAM_16 has no rLeakage data !
AIDREAM_18 has no OEF data !
AIDREAM_18 has no Delay data !
AIDREAM_18 has no COV data !
AIDREAM_18 has no rCMRO2 data !
AIDREAM_18 has no rLeakage data !
AIDREAM_19 has no OEF data !
AIDREAM_19 has no Delay data !
AIDREAM_19 has no COV data !
AIDREAM_19 has no rCMRO2 data !
AIDREAM_19 has no rLeakage data !
AIDREAM_25 has no 

In [17]:
list_patients_interim = list(set(list_patients_interim)) 
list_patients_interim = sort_name_list(list_patients_interim)

print(fr"Number of patients with available CERCARE data (for now) : {len(list_patients_interim)}")


Number of patients with available CERCARE data (for now) : 107


In [18]:
with open("list_intermediary_patients.txt", "w") as f:
    f.writelines([f"{p}\n" for p in list_patients_interim])
    

# 4. Load the Labels data : 

In [19]:
remaining_patients = {
    "pre_RT" : set(),
    "Rechute": set()
}


In [20]:
patients_with_labels = {
    "pre_RT" : set(),
    "Rechute": set()
}


## 4.1. Load the pre_RT Labels data :  

In [21]:
new_labels_src_dirs = {
    
    'pre_RT' : constants.DIR_AIDREAM_ON_NAS / 'Export_RayStation_RS_27062024' / 'preRT',
    'Rechute': constants.DIR_AIDREAM_ON_NAS / 'Export_RayStation_RS_27062024' / 'Rechute'
    
}


In [22]:
for stage in ['pre_RT', 'Rechute']:

    dir_src_labels = new_labels_src_dirs[stage]
    
    for folder in tqdm(os.listdir(dir_src_labels)):

        try:
            patient = folder.split("__")[-1]
            print(fr" ---------- {patient:^12} ----- {stage:^7} ---------- ")
    
            dir_src = dir_src_labels / folder
    
            list_labels = list(dir_src.glob("*.dcm"))
            
            if len(list_labels) != 1:
                print(fr"Conflicling Labels DICOM files ! ")
                
            else:
                path_src = list_labels[0]
                
                dir_dst = dir_aidream_data / patient / stage / 'Labels'
    
                if dir_dst.exists():
                    print(fr"Replacing {stage} Labels for patient {patient} !")
                    shutil.rmtree(dir_dst)
    
                else:
                    print(fr"Adding {stage} Labels for patient {patient} !")
                    
                dir_dst.mkdir(exist_ok=True, parents=True)
                path_dst = dir_dst / path_src.name
                
                print(fr"Path src exists: {path_src.exists()}")
                print(fr"Path parent dst exists: {path_dst.parent.exists()}")
                print(fr"Path dst exists: {path_dst.exists()}")

                shutil.copyfile(src=path_src, dst=path_dst)
                print(fr"Successfully loaded {stage} labels for patient {patient}")

        except Exception as e:
            print(e)
            print(fr'Failed to load {stage} Labels for patient {patient} !')

    

  0%|                                                                                                                                                                                       | 0/43 [00:00<?, ?it/s]

 ---------- AIDREAM_114  ----- pre_RT  ---------- 
Replacing pre_RT Labels for patient AIDREAM_114 !
Path src exists: True
Path parent dst exists: True
Path dst exists: False
Successfully loaded pre_RT labels for patient AIDREAM_114
 ---------- AIDREAM_152  ----- pre_RT  ---------- 
Replacing pre_RT Labels for patient AIDREAM_152 !
Path src exists: True
Path parent dst exists: True
Path dst exists: False


  5%|████████▏                                                                                                                                                                      | 2/43 [00:00<00:03, 11.85it/s]

Successfully loaded pre_RT labels for patient AIDREAM_152
 ---------- AIDREAM_135  ----- pre_RT  ---------- 
Replacing pre_RT Labels for patient AIDREAM_135 !
Path src exists: True
Path parent dst exists: True
Path dst exists: False


  9%|████████████████▎                                                                                                                                                              | 4/43 [00:00<00:05,  6.64it/s]

...

Successfully loaded Rechute labels for patient AIDREAM_118
 ---------- AIDREAM_129  ----- Rechute ---------- 
Replacing Rechute Labels for patient AIDREAM_129 !
Path src exists: True
Path parent dst exists: True
Path dst exists: False
Successfully loaded Rechute labels for patient AIDREAM_129
 ---------- AIDREAM_120  ----- Rechute ---------- 
Replacing Rechute Labels for patient AIDREAM_120 !
Path src exists: True
Path parent dst exists: True
Path dst exists: False


 95%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌         | 35/37 [00:06<00:00,  6.43it/s]

Successfully loaded Rechute labels for patient AIDREAM_120
 ---------- AIDREAM_232  ----- Rechute ---------- 
Replacing Rechute Labels for patient AIDREAM_232 !
Path src exists: True
Path parent dst exists: True
Path dst exists: False
Successfully loaded Rechute labels for patient AIDREAM_232
 ---------- AIDREAM_115  ----- Rechute ---------- 
Replacing Rechute Labels for patient AIDREAM_115 !
Path src exists: True
Path parent dst exists: True
Path dst exists: False


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 37/37 [00:06<00:00,  5.29it/s]

Successfully loaded Rechute labels for patient AIDREAM_115


In [23]:
for patient in tqdm(list_patients):
    for stage in ['pre_RT', 'Rechute']:

        dir_labels = dir_aidream_data / patient / stage / 'Labels'

        if len(list(dir_labels.glob("*.dcm"))) == 1:
            patients_with_labels[stage].add(patient)    
        else:
            remaining_patients[stage].add(patient)
        

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 234/234 [00:00<00:00, 51637.14it/s]


In [24]:
print(fr"Number of patients with full pre_RT imaging : {len(patients_with_labels['pre_RT'])}")
print(fr"Number of patients with full Rechute imaging : {len(patients_with_labels['Rechute'])}")


Number of patients with full pre_RT imaging : 184
Number of patients with full Rechute imaging : 143


In [28]:
print(fr"Number of interim patients : {len(list_patients_interim)}")

print(fr"Number of interim patients with full pre_RT imaging : {len(set(list_patients_interim).intersection(patients_with_labels['pre_RT']))}")

print(fr"Number of interim patients with full Rechute imaging : {len(set(list_patients_interim).intersection(patients_with_labels['Rechute']))}")


Number of interim patients : 107
Number of interim patients with full pre_RT imaging : 107
Number of interim patients with full Rechute imaging : 107
